In [7]:
from dataclasses import dataclass
from typing import List, Dict
from enum import Enum, auto

class CategoriaPlato(Enum):
    ENTRADA = auto()
    PRINCIPAL = auto()
    POSTRE = auto()
    BEBIDA = auto()

@dataclass
class Plato:
    """Clase que representa un plato del menú (Abstracción y Encapsulamiento)"""
    nombre: str
    precio: float
    categoria: CategoriaPlato

    def __str__(self):
        return f"{self.nombre} - ${self.precio:.2f}"

class Menu:
    """Clase que maneja el menú del restaurante (Responsabilidad Única)"""
    def __init__(self):
        self._platos = [
            Plato("Ensalada César", 8.50, CategoriaPlato.ENTRADA),
            Plato("Sopa del día", 6.00, CategoriaPlato.ENTRADA),
            Plato("Filete Mignon", 22.90, CategoriaPlato.PRINCIPAL),
            Plato("Pasta Alfredo", 15.50, CategoriaPlato.PRINCIPAL),
            Plato("Pollo a la Parrilla", 14.75, CategoriaPlato.PRINCIPAL),
            Plato("Tiramisú", 7.25, CategoriaPlato.POSTRE),
            Plato("Flan", 5.50, CategoriaPlato.POSTRE),
            Plato("Agua Mineral", 2.50, CategoriaPlato.BEBIDA),
            Plato("Refresco", 3.00, CategoriaPlato.BEBIDA),
            Plato("Vino Tinto", 8.00, CategoriaPlato.BEBIDA)
        ]

    @property
    def platos(self) -> List[Plato]:
        return self._platos

    def obtener_por_categoria(self, categoria: CategoriaPlato) -> List[Plato]:
        """Filtra platos por categoría (Abierto/Cerrado)"""
        return [plato for plato in self._platos if plato.categoria == categoria]

class ItemPedido:
    """Clase para manejar items del pedido (Composición)"""
    def __init__(self, plato: Plato, cantidad: int = 1):
        self.plato = plato
        self.cantidad = cantidad

    def subtotal(self) -> float:
        return self.plato.precio * self.cantidad

    def __str__(self):
        return f"{self.cantidad}x {self.plato.nombre} - ${self.subtotal():.2f}"

class Reservacion:
    """Clase principal de reservación (Encapsulamiento)"""
    def __init__(self, cliente: str, mesa: str):
        self.cliente = cliente
        self.mesa = mesa
        self._pedido: List[ItemPedido] = []

    def agregar_item(self, plato: Plato, cantidad: int = 1):
        """Agrega items al pedido"""
        self._pedido.append(ItemPedido(plato, cantidad))

    def calcular_total(self) -> float:
        return sum(item.subtotal() for item in self._pedido)

    def mostrar_resumen(self):
        """Muestra el resumen del pedido (Liskov Substitution)"""
        print(f"\n--- RESERVACIÓN PARA {self.cliente.upper()} ---")
        print(f"Mesa: {self.mesa}\n")
        print("Detalle del pedido:")
        for item in self._pedido:
            print(f"- {item}")
        print(f"\nTOTAL: ${self.calcular_total():.2f}")

class Restaurante:
    """Clase facade para gestionar la operación (Facade Pattern)"""
    def __init__(self):
        self.menu = Menu()
        self.reservas: Dict[str, Reservacion] = {}

    def nueva_reserva(self):
        """Método principal para crear reservas"""
        print("\n=== NUEVA RESERVACIÓN ===")
        cliente = input("Nombre del cliente: ").strip()
        mesa = input("Número de mesa: ").strip()

        reserva = Reservacion(cliente, mesa)
        self._seleccionar_platos(reserva)

        self.reservas[cliente] = reserva
        reserva.mostrar_resumen()

    def _seleccionar_platos(self, reserva: Reservacion):
        """Método helper para selección de platos"""
        print("\nSeleccione platos (ingrese número):")

        while True:
            print("\nCategorías disponibles:")
            for i, categoria in enumerate(CategoriaPlato):
                print(f"{i+1}. {categoria.name}")
            print("0. Terminar pedido")

            try:
                opcion = int(input("Opción: "))
                if opcion == 0:
                    break

                categoria_selec = list(CategoriaPlato)[opcion-1]
                platos = self.menu.obtener_por_categoria(categoria_selec)

                print(f"\nPlatos de {categoria_selec.name}:")
                for i, plato in enumerate(platos):
                    print(f"{i+1}. {plato}")

                plato_opcion = int(input("Seleccione plato: "))
                cantidad = int(input("Cantidad: ") or 1)

                reserva.agregar_item(platos[plato_opcion-1], cantidad)
                print(f"\n¡{platos[plato_opcion-1].nombre} agregado!")

            except (ValueError, IndexError):
                print("Opción inválida, intente nuevamente.")

if __name__ == "__main__":
    sistema = Restaurante()
    sistema.nueva_reserva()


=== NUEVA RESERVACIÓN ===

Seleccione platos (ingrese número):

Categorías disponibles:
1. ENTRADA
2. PRINCIPAL
3. POSTRE
4. BEBIDA
0. Terminar pedido

Platos de PRINCIPAL:
1. Filete Mignon - $22.90
2. Pasta Alfredo - $15.50
3. Pollo a la Parrilla - $14.75

¡Filete Mignon agregado!

Categorías disponibles:
1. ENTRADA
2. PRINCIPAL
3. POSTRE
4. BEBIDA
0. Terminar pedido

Platos de POSTRE:
1. Tiramisú - $7.25
2. Flan - $5.50

¡Flan agregado!

Categorías disponibles:
1. ENTRADA
2. PRINCIPAL
3. POSTRE
4. BEBIDA
0. Terminar pedido

Platos de BEBIDA:
1. Agua Mineral - $2.50
2. Refresco - $3.00
3. Vino Tinto - $8.00

¡Refresco agregado!

Categorías disponibles:
1. ENTRADA
2. PRINCIPAL
3. POSTRE
4. BEBIDA
0. Terminar pedido

--- RESERVACIÓN PARA DANIEL ---
Mesa: 7

Detalle del pedido:
- 1x Filete Mignon - $22.90
- 2x Flan - $11.00
- 1x Refresco - $3.00

TOTAL: $36.90
